Script obtained from: https://forums.fast.ai/t/google-image-scraper/79682

In [56]:
# search_term = [
# #     'agaricus moelleri',
#  'agaricus xanthodermus',
#  'amanita gemmata',
#  'clitocybe dealbata',
#               ]
search_term = [
    'agaricus arvensis',
 'agaricus augustus',
 'albatrellus ovinus',
 'armillaria mellea'
]
#  'clitocybe nebularis',
#  'coprinus picaceus',
#  'echinoderma asperum',
#  'helvella crispa',
#  'helvella lacunosa',
#  'hygrophoropsis aurantiaca',
#  'inocybe erubescens',
#  'inocybe geophylla',
#  'lactarius pubescens',
#  'lactarius torminosus',
#  'lepiota cristata',
#  'leucocybe connata',
#  'mycena rosea',
#  'russula fellea',
#  'russula nobilis',
#  'scleroderma citrinum',
#  'stropharia caerulea',
#  'tricholoma terreum',
#  'amanita muscaria',
#  'amanita pantherina',
#  'amanita phalloides',
#  'amanita porphyria',
#  'amanita regalis',
#  'amanita rubescens',
#  'amanita virosa',
#  'boletus satanas',
#  'clathrus ruber',
#  'coprinopsis atramentaria',
#  'cortinarius orellanus',
#  'cortinarius rubellus',
#  'cortinarius semisanguineus',
#  'galerina marginata',
#  'gyromitra esculenta',
#  'gyromitra infula',
#  'hebeloma crustuliniforme',
#  'hebeloma mesophaeum',
#  'hypholoma fasciculare',
#  'inocybe lacera',
#  'lactarius helvus',
#  'macrolepiota  rachodes',
#  'mycena pura',
#  'paxillus involutus',
#  'russula emetica',
#  'stropharia hornemannii']

In [49]:
DRIVER_PATH = r'C:/Users/catem/Desktop/chromedriver_win32/chromedriver'
number_images = 100
target_path = r'C:/Users/catem/data-science-projects/capstones/final_capstone/test'

In [50]:
!pip install selenium
from selenium import webdriver

In [51]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [52]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
        

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [53]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [57]:
import os 
import time
import requests
import io
from PIL import Image
import hashlib

for ele in search_term:
    search_and_download(search_term = ele, driver_path=DRIVER_PATH, target_path=target_path, number_images=number_images)

Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
SUCCESS - saved https://i.pinimg.com/originals/f3/02/e4/f302e4fd9430a13415f9edfdd159e21e.jpg - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_arvensis\5e059a2891.jpg
SUCCESS - saved https://c8.alamy.com/comp/X75HTW/agaricus-arvensis-X75HTW.jpg - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_arvensis\cb48a4a8b8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQCm5dRoikg2yc-jZKhc4hJX2jMvObNtS4Vnw&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_arvensis\5e384cca94.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTJWv-kAftLvyTWDK1PdPhsKiPA3IIyyppe4A&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_arvensis\42f4f35bdb.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSnEv4wEGZl34oJ5KO__aGf

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR5pZIOB6N_CeNghrlacwYRbGLRXRfJbV5ZMw&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_arvensis\5228477366.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQi3TTVufrc4NpDzhmmITjZjzGQcNTt8i_0A&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_arvensis\f91b3c8822.jpg
SUCCESS - saved https://ultimate-mushroom.com/uploads/posts/2020-10/1601544890_agaricus-arvensis-2.jpg - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_arvensis\e2582a7c59.jpg
SUCCESS - saved https://mycoportal.org/dbg/mycology/00018/18030_F00.jpg - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_arvensis\6d5961f0f9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSIANvJMJv6r10kgLjGPXLPbw4RtkMbWQdVvQ&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTvmYLGsBdko6RT50FiyDC5Hk1N-qcRvMeLDg&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_arvensis\259c51ea23.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSNXkdFY8sI4HBhYvlYHp628CU3Mf9foMT1oA&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_arvensis\ed1d38d3a9.jpg
SUCCESS - saved https://previews.123rf.com/images/ekina/ekina1704/ekina170411805/75766192-agaricus-arvensis-horse-mushroom-genus-agaricus-.jpg - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_arvensis\ef41e74c86.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQLm5lkSqj9cYq1B888mst1uxujX9_l3GTonQ&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_arvensis\89da468891.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQlVbQ7CTT1YKKFYkunP

SUCCESS - saved https://ultimate-mushroom.com/uploads/posts/2020-09/1600022502_agaricus-augustus-3.jpg - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_augustus\e07741664e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTs3tsOLXulcY8n6SQTgLkr6C8kwFKfc5ojBw&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_augustus\504a586c11.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRzGLQU9o8NVQ_N0onWUXgYHH7qt92FFnUXKg&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_augustus\a6c4ea3b93.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR13_8oikhmcVQ51qzmqsh-5xBAttpZAE0i1w&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_augustus\eeefd4762d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQRN-oNk-m-FNvYYSyBD_R9Fvd2zmj03dQxhA&usqp=CAU - as C:/Users

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQw-q7W-oOf-qj9k3SMRRhfHvfBy_A142SOuQ&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_augustus\45bf436591.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTMnmPgm1CUkGxTxJ6qsLKWiFioJqS52rwXjQ&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_augustus\0b7b8277ba.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ46baHAhMts8xOJFeEhdcxzNs8ue0AfW6IUQ&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_augustus\668d9e7061.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQUynuvHv4lkfU9PzVPxGK6L9MbRsOZrB94-g&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\agaricus_augustus\832950b811.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQvTW1OKnBdnpx_-FdJv7CV3JUCT86E3qp4w&usqp=CAU 

...

SUCCESS - saved https://www.fungikingdom.net/_Media/albatrellus-ovinus-1010596_med_hr.jpeg - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\albatrellus_ovinus\eebca65da0.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRP5PI94CaLUyvMd8URB4D-XwItKWKlcvNURA&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\albatrellus_ovinus\85ee13e095.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/d/d2/Albatrellus_ovinus_3.jpg - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\albatrellus_ovinus\a8c97d8238.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQhPcMkvpZWwu63OoPMTqjW5EOHojU4KFTcHA&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\albatrellus_ovinus\dc4567e219.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTv_5qXwDn3QwOLRXuS5TyCnLTE7aoJn4Nfrg&usqp=CAU - as C:/Users/catem/data-science-projects/ca

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQoK88N0YE4CJk9uuV-CcTGyPGZq9kEUmKpJQ&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\albatrellus_ovinus\a2960d4498.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYHaT3UiS09Hg6gFu8aQLqbyeZdyZmGBeH9A&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\albatrellus_ovinus\6080fa3232.jpg
SUCCESS - saved https://cryofthebolete.files.wordpress.com/2018/11/albatrellus-ovinus-2.jpg - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\albatrellus_ovinus\25a86bc874.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcREP1cDiMtKTZ_gMIcG5D_5fR_iwwG88QNrrQ&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\albatrellus_ovinus\7d47d38a79.jpg
SUCCESS - saved https://media.istockphoto.com/photos/albatrellus-ovinus-edible-mushroom-picture-id486621174?k=6&m=486621174&s=170667a&w=

Found: 100 search results. Extracting links from 0:100
Found: 101 image links, done!
SUCCESS - saved https://media.istockphoto.com/photos/armillaria-mellea-mushroom-picture-id1044904816 - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\armillaria_mellea\b2d952e942.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSqfoHIpV6d7wZvLstglqBEJ_WQtjO_059mPw&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\armillaria_mellea\8de14f726b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTdYe-ltc4pLlfqz7R72Bi6yPFZUxmHwnEcXw&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\armillaria_mellea\ef5875ece9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRVHg7ZoLmjfJuWP-CadSQWFHEFwoUCPs7D-A&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\armillaria_mellea\f7c90a6ff0.jpg
SUCCESS - saved https://www.messiah.edu/Oakes/

SUCCESS - saved https://www.mushroomexpert.com/images/kuo6/armillaria_mellea_03.jpg - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\armillaria_mellea\9451405315.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSTebZtcP-EwqRMXr4KuZ3vVbRADexujJu6aA&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\armillaria_mellea\8ac747863b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTYuhRz1NgMSpwDOgWIRP5dnMCmRumZLvODog&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\armillaria_mellea\6f2644aad1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRVYhERXMa5kIvN3BR1-Ebo9kbMzTpjJ-XWfw&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\armillaria_mellea\d14f0c40b3.jpg
SUCCESS - saved https://i.pinimg.com/originals/cc/3e/ae/cc3eaef7ac40317d64b0e3b0a966801c.jpg - as C:/Users/catem/data-science-projects/capstones/fin

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTCoFdIqMQ3tI0v--L-d-LEK5rYmpEK1Du_oA&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\armillaria_mellea\d667c4cfb4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQgHmMDNebLR5XcK4scV2RUUG56dwwQWE7rYw&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\armillaria_mellea\b10360cc92.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSvD4SXsOIwXBF9BdFwJT0vKEsJeIcVSabu7w&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\armillaria_mellea\a4ebbcff3f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQDCHoPu9zdxcJX3cIYI8DL9kvA0YfuKW8QiQ&usqp=CAU - as C:/Users/catem/data-science-projects/capstones/final_capstone/test\armillaria_mellea\e22c6d0f4b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQjGDv3PsH6FeDk3Eflk8y1Ms7obqTSPQpsHw&usqp=CAU 